# Scene Creation

The aim of this notebook is to create a scene with chosen camera intrinsic and extrinsic parameters.

A scene consists of the following elements:

- Camera Frames
- Bounding Box Labels
- Center of Mass Ground Truth

[*visualization*](https://lucid.app/lucidchart/a570a6c2-e5ef-412b-86fb-98b6d1292aa6/edit?viewport_loc=138%2C-10%2C1707%2C743%2C0_0&invitationId=inv_d45a706f-3778-4e32-a766-520a623f508f)


***TODO:***
- capire come trattare l'origine del sistema di riferimento SinD rispetto al nostro
- cambiare il modo in cui definiamo la camera position
- se azimuth ed elevation li calcoliamo a partire dai camera parameters e dalla posizione del centro della bounding box nell'image plane, perchè li dobbiamo usare nella regressione? -> vuol dire che dobbiamo cambiare la classe Dataset in modo che due parametri vengano presi dal dataset e due vengano calcolati e 4 elementi siano dati in input alla rete.

In [ ]:
import numpy as np
from utils import *
np.random.seed(42)

## Scene Parameters Setting

In [ ]:
# Define Camera Intrinsic Parameters
focal_length=0.0036
sensor_size=(0.00367, 0.00274)
image_size=(640, 480)
kappa=0.4


# Define Camera Position
alpha, beta = random_view()                     # alpha, beta are respectively azimuth and elevation for the camera orientation
camera_distance = random.uniform(20, 40)
camera_position = camera_distance * np.array([
    np.cos(beta) * np.cos(alpha),
    np.cos(beta) * np.sin(alpha),
    np.sin(beta)
])

# Define Camera Orientation: the camera always points at the origin of the world coordinates
up = np.array([0, 0, 1])                   # define positive vertical direction
from_point = camera_position        # define 'from' and 'to' points for the camera
to_point = np.array([0, 0, 0])
R_C2W, t_C2W = lookat(from_point, to_point, up)     # compute Rotation and Translation -> Transformation Matrix
R_C2W = R_C2W @ matrix_from_axis_angle((1, 0, 0, np.pi))     # flips about axis 1 to obtain Camera Frame
cam2world = transform_from(R_C2W, t_C2W)
intrinsic_camera_matrix = np.array([                # create intrinsic camera matrix
    [focal_length, 0, sensor_size[0] / 2],
    [0, focal_length, sensor_size[1] / 2],
    [0, 0, 1]
])

## Load Data from SinD

Dobbiamo prendere tutto quello che ci interessa e che ci servirà dopo per creare i labels.

## *Bounding Box Labels*

It will generate the json file annotations, which will look something like this:

```json
"annotations": [
        {
            "area": 4.292386303916224,
            "bbox": [
                1217.57,
                335.11,
                8.43,
                10.35
            ],
            "category_id": 2,
            "id": 163514,
            "image_id": 1,
            "iscrowd": 0,
            "point": [
                270.0,
                75.0
            ],
            "segmentation": []
        },
```

Qui si userà ```generate_box()``` e tutto il resto per generare le bounding box.

- prendi la posizione
- prendi la dimensione
- prendi l'orientamento

-> hai i vertici della box nelle world coordinates

- converti in 2D coordinates on image plane
- crea bounding box on image plane con `create_bounding_box()`

## *Save Trajectory Ground Truth*

It will save the trajectory of the center of mass from SinD data. There are two remarks:

- it could be included into the json file above
- it needs to include some sort of id identifiers in the case of multiple vechiles

## *Blender Rendering*

It will run the python script to load the scene in Blender (setting keyframes, camera parameters, etc.) and render the images.

It will all be done by running python script in Blender via CLI.

***Che ci deve stare nelle annotations?***

per ogni frame:

- object_id
- class_id -> ['car', 'truck', 'bus', 'motorcycle', 'bicycle', 'pedestrian']
- bbox -> [x, y, w, h]

<frame_id> <object_id> <bbox_x> <bbox_y> <bbox_width> <bbox_height> <class_id> <conf> <xc_true> <yc_true> <xw_true> <yw_true> <zw_true> <yaw_rad> ...